In [1]:
# required modules
import random
import json
import pickle
import nltk
import numpy as np
from keras.models import load_model
from nltk.stem import WordNetLemmatizer
  
lemmatizer = WordNetLemmatizer()
intents = json.loads(open("intents.json").read())
words = pickle.load(open("words.pkl", 'rb'))
classes = pickle.load(open("classes.pkl", 'rb'))
model = load_model("chatbot_model.h5")

In [2]:
def clean_up_sentence(sentence):
    # tokenize the pattern - split words into array
    sentence_words = nltk.word_tokenize(sentence)
    # stem each word - create short form for word
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words

def bag_of_words(sentence):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    bag = [0] * len(words)
    for s in sentence_words:
        for i, w in enumerate(words):
            # check if the word is in the vocabulary
            if w == s:
                # assign 1 if current word is in the vocabulary position
                bag[i] = 1
    # return bag of words array
    return np.array(bag)

def predict_class(sentence):
    # filter below threshold predictions
    p = bag_of_words(sentence)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.5
    results = [[i, r] for i, r in enumerate(res) if r > ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
        return return_list
        
def get_response(intents_list, intents_json):
    tag = intents_list[0]['intent']
    list_of_intents = intents_json['intents']
    result = ""
    for i in list_of_intents:
        if i['tag'] == tag:
            result = random.choice(i['responses'])
            break
    return result
  
print("Chatbot is up!")


Chatbot is up!


In [3]:
print("You have been connected with GarBot 1.0 (type quit to stop)!")
Lucas = True
while Lucas:
    message = input("")
    if message.lower() == "quit":
        break
    ints = predict_class(message)
    res = get_response(ints, intents)
    print(res)

Start talking with the bot (type quit to stop)!
1/1 [==============================] - 1s 708ms/step
Hello, thanks for visiting
1/1 [==============================] - 0s 92ms/step
Sounds great! I've sent your order to Nichole and Robert and you will receive an order confirmation email soon!
1/1 [==============================] - 0s 94ms/step
Our Logistics department will be in touch with you shortly!
